# P1
Written by Babak Hosseini Parsa Ahmadi 810101609, Mohtasham 810101408 and Mahdy Naeini 810101536\
Data science project abou digikala mobile phones.

In [1]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import scipy.stats as stats
import numpy  as np
import seaborn as sns
import datetime
import time
import re

# Scraping DigiKala

In [2]:
SUB_FEATURES = {"مدل": "Model", "زمان معرفی": "Release Data","وزن": "Weight", "ابعاد": "Dimensions", "برند": "Brand","فروشنده":"Seller",
                "توضیحات سیم کارت":"Sim Card Description", "نوع سیم کارت":"Sim Card Type", "ساختار بدنه":"Body Structure",
                "ویژگی‌های کلیدی": "Key Features", "پردازنده‌ مرکزی": "CPU", "فرکانس پردازنده‌ مرکزی":"CPU Clock Rate",
                "پردازنده‌ گرافیکی":"GPU","توضیحات کارت حافظه جانبی":"SD Card Description","بازه‌ اندازه صفحه نمایش":"Range of Screen Size",
                "نسبت تصویر":"DAR","سایر قابلیت‌ها":"Other Screen Features","موقعیت نما ( GPS )":"GPS","نوع Wi-Fi (ورژن)":"Wi-Fi Type",
                "بلوتوث":"Bluetooth","نسخه بلوتوث":"Bluetooth Version","درگاه‌های ارتباطی":"Ports","رزولوشن عکس":"Camera Resolution",
                "فناوری فوکوس":"Focus Technology","فلش":"Flashlight","قابلیت‌های دوربین":"Camera Features","فیلمبرداری":"Video Recording",
                "خروجی صدا":"Audio Output","سیستم عامل":"OS","نسخه سیستم عامل":"OS Version","حس‌گرها":"Sensors",
                "مشخصات باتری": "Battery Spec","اقلام همراه":"Accessories","سایر توضیحات اقلام همراه":"Accessories Description",
                "دسته ‌بندی": "Category", "تراشه":"Chipset","حافظه داخلی":"Internal Memory","مقدار RAM":"RAM","گارانتی":"Warranty",
                "پشتیبانی از کارت حافظه": "SD Card Support", "فناوری صفحه‌ نمایش": "Display Technology", "قیمت نهایی": "Price Final",
                "اندازه":"Screen Size","رزولوشن صفحه نمایش":"Screen Resolution","تراکم پیکسلی":"Pixel Density","توضیحات تکمیلی صدا":"Supplementary Audio Description",
                "نسبت صفحه‌ نمایش به بدنه": "Ratio of Screen Size to Body", "شبکه‌های ارتباطی": "Cellular Network",
                "شبکه‌های مخابراتی و ارتباطی قابل پشتیبانی":"Supported Networks","دوربین‌های پشت گوشی":"Back Facing Cameras",
                "دوربین سلفی": "Front Facing Cameras", "رنگ های موجود": "Available Colors", "قیمت بدون تخفیف": "Price No Discount",
                "ویژگی‌های خاص": "Special Features","تعداد سیم کارت":"Number of Sim Cards","امتیاز خریداران":"Score"}

In [3]:
def make_new_csv(file_addr):
    with open(file_addr, mode='w') as mobiles_file:
        mobilesCSV = csv.writer(mobiles_file)
        mobilesCSV.writerow(list(SUB_FEATURES.values()))

In [ ]:
def DK_extract_features(addr, driver,writer):
    driver.get("https://www.digikala.com/"+addr)
    sleep(1)
    spec_button=None
    spec_more_button=None
    while True:
        try:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(1)
            driver.execute_script('window.scrollTo(0, -document.body.scrollHeight);')
            spec_button = driver.find_element(By.ID, "specification").find_elements(
                By.XPATH, "./..")
            sleep(1)
            driver.execute_script("arguments[0].scrollIntoView();", spec_button[0])
            sleep(1)
            spec_more_button=spec_button[0].find_elements(By.TAG_NAME, "span")
            spec_more_button[-2].click()
            sleep(1)
            break
        except (NoSuchElementException, ElementClickInterceptedException):
            continue
    page = driver.page_source
    soup = bs(page, 'html.parser')

    global SUB_FEATURES
    scraped_sub_features = dict()
    features_div=soup.find(string="مشخصات کلی").parent.parent.parent
    for fa_feature, eng_feature in (SUB_FEATURES.items()):
        if features_div.find(string=fa_feature):
            found=features_div.find(
                string=fa_feature).parent.parent.findAll('p')[1:]
            scraped_sub_features[eng_feature] = "\r\n".join([s.text for s in found])
        else:
            scraped_sub_features[eng_feature] = ""

    colors_element=soup.find("div", {"class": "flex lg:flex-wrap overflow-x-auto px-5 lg:px-0"}
            ).findAll("div", {"class": "whitespace-nowrap"})
    colors=[]
    for color in colors_element:
        colors.append(color.text)
    scraped_sub_features['Available Colors'] = colors

    score = soup.findAll("p", {"class": "mr-1 text-body-2"})
    if score:
        scraped_sub_features["Score"] = score[0].text
    
    brand = soup.findAll("a", {"data-cro-id": "pdp-breadcrumb-down"})
    if brand:
        scraped_sub_features["Brand"] = brand[0].text

    sellers=(soup.find("div", {"id": "sellerSection"}))
    if not sellers:
        price_no_discount = soup.findAll("span", {"data-testid": "price-no-discount"})
        for element in price_no_discount:
            if element and " ".join(element.parent.parent.parent.attrs["class"][0:2]) == "flex items-center":
                scraped_sub_features["Price No Discount"] = element.text
                break

        price_final = soup.findAll("span", {"data-testid": "price-final"})
        for element in price_final:
            if element and " ".join(element.parent.parent.parent.attrs["class"][0:2]) == "flex items-center":
                scraped_sub_features["Price Final"] = element.text
                break

        Warranty = soup.findAll(string=re.compile("گارانتی"))
        if Warranty:
            scraped_sub_features["Warranty"] = Warranty[0].parent.text
            
        scraped_sub_features["Seller"] = soup.find('h3', string=re.compile("فروشنده")).parent.parent.findChildren(
            "div", recursive=False)[2].findChildren("p", recursive=True)[0]
        writer.writerow(list(scraped_sub_features.values()))
        return

    sellers_button = driver.find_element(By.ID, "sellerSection")
    driver.execute_script(
        "arguments[0].scrollIntoView();", sellers_button)
    sleep(1)
    sellers_more_button = sellers_button.find_elements(
        By.TAG_NAME, "span")
    if sellers_more_button:
        sellers_more_button[-1].click()

    page = driver.page_source
    soup = bs(page, 'html.parser')
    sellers = (soup.find("div", {"id": "sellerSection"}))
    for seller in sellers.findChildren("div",recursive=False)[0].\
        findChildren("div",recursive=False)[1:]:
        seller_div=seller.findChildren("div", recursive=False)[0].findChildren("div", recursive=False)
        name=seller_div[0].findChildren("div", recursive=False)[1]
        scraped_sub_features["Seller"] = name.findChildren(
            "div", recursive=False)[1].findChildren("div", recursive=False)[0].text
        warranty=seller_div[0].findChildren("div", recursive=False)[3]
        scraped_sub_features["Warranty"] = warranty.text
        price = seller_div[1].findChildren(
            "div", recursive=False)[0].findChildren(
            "div", recursive=False)[0].findChildren(
            "div", recursive=False)[0]
        prices=price.findChildren(
            "span", recursive=True)
        scraped_sub_features["Price No Discount"] = prices[-1].text
        if len(prices)==3:
            scraped_sub_features["Price No Discount"] = prices[0].text
            scraped_sub_features["Price Final"] = prices[1].text
        writer.writerow(list(scraped_sub_features.values()))

In [5]:
def DK_scrape_list_of_mobiles(number_of_pages):
    driver = webdriver.Chrome()
    mobiles = []
    i = 1
    while i <= number_of_pages:
        driver.get(
            f'https://www.digikala.com/search/category-mobile-phone/product-list/?has_selling_stock=1&page={i}')
        wait = WebDriverWait(driver, 60)
        wait.until(EC.presence_of_element_located((
            By.XPATH, "//div[@data-testid='product-card']")))
        page = driver.page_source
        soup = bs(page, 'html.parser')
        mobiles.extend(soup.find_all(
            'div', {'class': 'product-list_ProductList__item__LiiNI'}))
        driver.execute_script(
            'window.scrollTo(0, document.body.scrollHeight);')
        sleep(1)
        i += 1
    driver.close()
    return mobiles

In [6]:
def DK_scrape_all_mobiles(file_addr,mobiles,start):
    driver = webdriver.Chrome()
    csv_file = open(file_addr, mode='a', encoding='utf-8')
    csv_writer=csv.writer(csv_file)
    i=start
    while i<len(mobiles):
        try:
            DK_extract_features(mobiles[i].find('a')['href'], driver, csv_writer)
            i+=1
        except (NoSuchElementException, ElementClickInterceptedException):
            print(i)
        except Exception:
            print(i)
            driver.close()
            csv_file.close()
            raise
    driver.close()
    csv_file.close()

In [7]:
FILE_ADDR='digikala_phones.csv'

In [8]:
#make_new_csv(FILE_ADDR)

In [9]:
#DK_mobiles = DK_scrape_list_of_mobiles(14)

In [15]:
DK_scrape_all_mobiles(FILE_ADDR, DK_mobiles,0)

# Preprocessing DigiKala

In [16]:
PREPROCESSING_MAP = {"دسامبر": 12, "نوامبر": 11, "اکتبر": 10, "سپتامبر": 9, "آگوست": 8, "جولای": 7, "ژوئن": 6, "می": 5, "آوریل": 4, "مارچ": 3, "مارس": 3,
                      "فوریه": 2, "ژانویه": 1,"دو":"2","سه":3,
                      "یک":"1",".":".","۰":"0","۱":"1","۲":"2","۳":"3","۴":"4","۵":"5","۶":"6","۷":"7","۸":"8","۹":"9"}

In [17]:
original_df=pd.read_csv(FILE_ADDR)
df=original_df.copy()

In [18]:
len(df)

773

In [19]:
df.drop_duplicates(inplace=True)
target_cols = df.columns
multi_cols = list(filter(lambda x: df[x].astype(str).str.contains("\r\n").any(), df.columns))
for col in target_cols:
    df.loc[:, col] = df[col].apply(lambda x: "".join(
        [PREPROCESSING_MAP[c] if c in PREPROCESSING_MAP else c for c in x]).replace("\u200d", " ") if x is not np.nan and type(x) is not float else x).astype(object)
    if col in multi_cols:
        df.loc[:, col] = df[col].apply(lambda x: x.replace("\u200c", " ").split(
            "\r\n") if x is not np.nan and type(x) is not float else x)
df[target_cols]

,Model,Release Data,Weight,Dimensions,Brand,Seller,Sim Card Description,Sim Card Type,Body Structure,Key Features,...,Ratio of Screen Size to Body,Cellular Network,Supported Networks,Back Facing Cameras,Front Facing Cameras,Available Colors,Price No Discount,Special Features,Number of Sim Cards,Score
0,Redmi Note 13 4G,15 ژانویه 2024,188.5 گرم,8×75.6×162.3 میلی‌متر,شیائومی,دیجی‌کالا,[سایز نانو (8.8 × 12.3 میلی متر) ],NaN,NaN,[رابط کاربری MIUI 14 ],...,87.5%,"[2G , 3G , 4G ]","[Wi-Fi , بلوتوث ]",2 ماژول دوربین,[دارای 1 حسگر دوربین | دوربین با رزولوشن 16 م...,"['مشکی', 'آبی', 'سبز']","9,699,000","[مجهز به حس گر اثرانگشت , مناسب بازی , مناسب ع...",دو عدد,4.4
1,Redmi Note 13 4G,15 ژانویه 2024,188.5 گرم,8×75.6×162.3 میلی‌متر,شیائومی,دیجی‌کالا,[سایز نانو (8.8 × 12.3 میلی متر) ],NaN,NaN,[رابط کاربری MIUI 14 ],...,87.5%,"[2G , 3G , 4G ]","[Wi-Fi , بلوتوث ]",2 ماژول دوربین,[دارای 1 حسگر دوربین | دوربین با رزولوشن 16 م...,"['مشکی', 'آبی', 'سبز']","9,799,000","[مجهز به حس گر اثرانگشت , مناسب بازی , مناسب ع...",دو عدد,4.4
2,Redmi Note 13 4G,15 ژانویه 2024,188.5 گرم,8×75.6×162.3 میلی‌متر,شیائومی,فیروزه همراه اسپادانامنتخب,[سایز نانو (8.8 × 12.3 میلی متر) ],NaN,NaN,[رابط کاربری MIUI 14 ],...,87.5%,"[2G , 3G , 4G ]","[Wi-Fi , بلوتوث ]",2 ماژول دوربین,[دارای 1 حسگر دوربین | دوربین با رزولوشن 16 م...,"['مشکی', 'آبی', 'سبز']","9,904,000","[مجهز به حس گر اثرانگشت , مناسب بازی , مناسب ع...",دو عدد,4.4
3,Redmi Note 13 4G,15 ژانویه 2024,188.5 گرم,8×75.6×162.3 میلی‌متر,شیائومی,اسمارت تکنولوژی قشم,[سایز نانو (8.8 × 12.3 میلی متر) ],NaN,NaN,[رابط کاربری MIUI 14 ],...,87.5%,"[2G , 3G , 4G ]","[Wi-Fi , بلوتوث ]",2 ماژول دوربین,[دارای 1 حسگر دوربین | دوربین با رزولوشن 16 م...,"['مشکی', 'آبی', 'سبز']","9,999,000","[مجهز به حس گر اثرانگشت , مناسب بازی , مناسب ع...",دو عدد,4.4
4,Redmi Note 13 4G,15 ژانویه 2024,188.5 گرم,8×75.6×162.3 میلی‌متر,شیائومی,اسمارت تکنولوژی قشم,[سایز نانو (8.8 × 12.3 میلی متر) ],NaN,NaN,[رابط کاربری MIUI 14 ],...,87.5%,"[2G , 3G , 4G ]","[Wi-Fi , بلوتوث ]",2 ماژول دوربین,[دارای 1 حسگر دوربین | دوربین با رزولوشن 16 م...,"['مشکی', 'آبی', 'سبز']","9,999,000","[مجهز به حس گر اثرانگشت , مناسب بازی , مناسب ع...",دو عدد,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768,Galaxy A55,11 مارس 2024,213 گرم,161.1x77.4x8.2 میلی‌متر,سامسونگ,سپاهان همراه یاقوتمنتخب,[سایز نانو (8.8 × 12.3 میلی متر) ],NaN,"[قاب پشتی از شیشه (Gorilla Glass) /, فریم آلوم...",[استاندارد IP67 (مقاومت در برابر نفوذ گرد و غب...,...,85.8%,"[2G , 3G , 4G , 5G ]",NaN,3 ماژول دوربین,"[با سنسور 32 مگاپیکسلی عریض f/2.2, 26mm 1/2., ...","['زرد', 'سرمه ای', 'یاسی', 'زرد لیمویی', 'آبی ...","17,720,000","[مجهز به حس گر اثرانگشت , مقاوم در برابر آب ]",دو عدد,4
769,Galaxy A55,11 مارس 2024,213 گرم,161.1x77.4x8.2 میلی‌متر,سامسونگ,فیروزه همراه اسپادانامنتخب,[سایز نانو (8.8 × 12.3 میلی متر) ],NaN,"[قاب پشتی از شیشه (Gorilla Glass) /, فریم آلوم...",[استاندارد IP67 (مقاومت در برابر نفوذ گرد و غب...,...,85.8%,"[2G , 3G , 4G , 5G ]",NaN,3 ماژول دوربین,"[با سنسور 32 مگاپیکسلی عریض f/2.2, 26mm 1/2., ...","['زرد', 'سرمه ای', 'یاسی', 'زرد لیمویی', 'آبی ...","17,912,000","[مجهز به حس گر اثرانگشت , مقاوم در برابر آب ]",دو عدد,4
770,Galaxy A55,11 مارس 2024,213 گرم,161.1x77.4x8.2 میلی‌متر,سامسونگ,فیروزه همراه اسپادانامنتخب,[سایز نانو (8.8 × 12.3 میلی متر) ],NaN,"[قاب پشتی از شیشه (Gorilla Glass) /, فریم آلوم...",[استاندارد IP67 (مقاومت در برابر نفوذ گرد و غب...,...,85.8%,"[2G , 3G , 4G , 5G ]",NaN,3 ماژول دوربین,"[با سنسور 32 مگاپیکسلی عریض f/2.2, 26mm 1/2., ...","['زرد', 'سرمه ای', 'یاسی', 'زرد لیمویی', 'آبی ...","18,608,000","[مجهز به حس گر اثرانگشت , مقاوم در برابر آب ]",دو عدد,4
771,Xiaomi Redmi Note 12 Turbo,28 مارس 2023,181 گرم,161.1X75X7.9 میلی‌متر,شیائومی,پایاگستر شهرمنتخب,[سایز نانو (8.8 × 12.3 میلی متر) ],NaN,[بخش جلو از شیشه / فریم پلاستیکی / بخش پشتی از...,[برای استفاده از گوشی به صورت گلوبال و پشتیبان...,...,88.9,"[2G , 3G , 4G , 5G ]","[NFC , OTG ]",3 ماژول دوربین,[دارای 1 حسگر دوربین | دوربین با رزولوشن 16 مگ...,"['مشکی', 'سفید', 'آبی']","18

In [ ]:
target_cols = ['Price No Discount', 'Price Final']
df['Price Final'].fillna(df['Price No Discount'], inplace=True)
for col in target_cols:
    df.loc[:, col] = df[col].apply(lambda x: int(
        x.replace(',', '') if x is not np.nan else x))
df[target_cols]

,Price No Discount,Price Final
0,37650000,37650000
1,5855000,5855000
2,4870000,4870000
3,1210000,1210000
4,7090000,7090000
...,...,...
278,8399000,8399000
279,4899000,4899000
280,4259000,4259000
281,18499000,18499000


In [ ]:
target_cols = ['Weight', 'Bluetooth Version', 'Camera Resolution']
for col in target_cols:
    df.loc[:, col] = df[col].apply(lambda x: (x.replace('%', '').replace(',',' ').split()[0]) if x is not np.nan else x)
df[target_cols]

,Weight,Bluetooth Version,Camera Resolution
0,174,5.0,12
1,200,5.3,50
2,194,5.1,50
3,78.7,NaN,NaN
4,192,5.3,50
...,...,...,...
278,193,5.0,48
279,208,5.0,13
280,204,5.0,50
281,181,5.3,64


In [ ]:
target_cols = ['X Dimension', 'Y Dimension', 'Z Dimension']
for i,col in enumerate(target_cols):
    df.loc[:, col] = df['Dimensions'].apply(lambda x: float("".join(x.split()[0:-1]).replace('*', '×').replace('X', '×').replace('x', '×').split('×')[i])
                                            if x is not np.nan else x)
df[target_cols]

In [ ]:
target_cols = ['Number of Sim Cards']
for col in target_cols:
    df.loc[:, col] = df[col].apply(lambda x: (
        PREPROCESSING_MAP[x.split()[0]]) if x is not np.nan else x)
df[target_cols]

,Number of Sim Cards
0,2
1,2
2,1
3,2
4,2
...,...
278,2
279,2
280,2
281,2


In [ ]:
target_cols = ['DAR']
for col in target_cols:
    df.loc[:, col] = df[col].apply(lambda x: round(float(
        x.replace(' ','').split(':')[0])/float(
        x.replace(' ', '').split(':')[1]), 3) if x is not np.nan else x)
df[target_cols]

In [ ]:
target_cols = ['X Screen Resolution',
               ' Y Screen Resolution']
for i, col in enumerate(target_cols):
    df.loc[:, col] = df['Screen Resolution'].apply(lambda x: 
                                                   (re.sub("[^0-9]"," ",x).strip(' ').split(' ')[-i])
                                                   if x is not np.nan else x)
df[target_cols]

,X Screen Resolution,Y Screen Resolution
0,1170,2532
1,1080,2340
2,1080,2400
3,120,160
4,720,1600
...,...,...
278,1080,2388
279,720,1600
280,720,1600
281,1080,2400


In [ ]:
target_cols = ['Release Data']
for col in target_cols:
    df.loc[:, col] = df[col].apply(lambda x: x.split()[2]+
                                   f", {PREPROCESSING_MAP[x.split()[1]]}" +
                                   f", {x.split()[0]}" if x is not np.nan else x)
df[target_cols]

In [ ]:
target_cols = ['Release Data UNIX']
for col in target_cols:
    df.loc[:, col] = df['Release Data'].apply(
        lambda x: time.mktime(datetime.date(int(x.split(', ')[0]), int(x.split(', ')[1]), int(x.split(', ')[2])).timetuple()) 
        if x is not np.nan else x)
df[target_cols]

In [ ]:
target_cols = ['Warranty']
for col in target_cols:
    df.loc[:, col] = df[col].apply(lambda x: x.split("ماهه")[1].split('(')[0].strip() if x is not np.nan else x)
df[target_cols]

In [ ]:
numerical_cols = df.select_dtypes(include=[np.number])
plt.figure(figsize=(10, 10))
sns.heatmap(numerical_cols.corr(), annot=True, cmap="Blues")
plt.title("Correlation Heatmap")
plt.show()

# Scraping Technolife

In [75]:
SUB_FEATURES = {"نوع پردازنده - CPU": "CPU", "تعداد هسته پردازشگر": "Number of Cores", "پردازنده گرافیکی - GPU": "GPU", "تعداد سیم کارت": "Number of Sim Cards", "سیستم عامل": "OS",
                "کیفیت دوربین": "Camera Resolution", "ابعاد/ وزن": "Weight/Height", "ساختار بدنه": "Body Structure", "تاریخ معرفی": "Release Data", "امکان شارژ از طریق USB":"Charge through USB",
                "حافظه داخلی": "Internal Memory", "حافظه RAM": "RAM", "امکان افزایش حافظه/ نوع کارت حافظه": "Memory Expandable/Type", "نوع صفحه نمایش": "Display Technology",
                "سایز صفحه نمایش": "Screen Size", "تعداد رنگ/ رزولوشن": "Screen Resolution", "درصد نسبت صفحه نمایش به بدنه": "Ratio of Screen Size to Body", "نسبت صفحه نمایش": "DAR",
                "تراکم پیکسل": "Pixel Density", "فیلمبرداری دوربین پشت": "Back Camera Recording", "فلاش": "Flashlight", "زوم دیجیتال": "Digital Zoom", "دوربین جلو":"Front Facing Cameras",
                "ویژگی‌ های مهم صفحه نمایش": "Display Features", "دوربین پشت": "Back Facing Cameras", "مشخصات سخت‌ افزاری دوربین": "Camera HardWare", "قابلیت USB On-The-Go یا OTG":"OTG",
                "سایر ویژگی‌ های مهم دوربین": "Camera Features", "شبکه اینترنت": "Cellular Network", "شبکه‌ های مخابراتی قابل پشتیبانی": "Supported Networks", "پورت USB/ ورژن و نوع": "USB PORT",
                "بلوتوث/ ورژن و ویژگی":"Bluetooth","موقعیت‌ نما GPS/ ویژگی‌ ها":"GPS and features","سنسورها":"Sensors","سایر مشخصات مهم":"Special Features","ظرفیت باتری":"Battery Capacity",
                "نوع باتری/ قابل تعویض توسط کاربر":"Battery Type","قابلیت شارژ سریع":"Fast Charge","فرکانس پردازنده مرکزی":"CPU Frequency","میزان روشنایی صفحه نمایش":"Screen Brightness",
                "نرخ تازه‌ سازی":"Screen Refresh Rate","محافظ صفحه نمایش":"Screen Glass"}

TODO:
Colors and beyond

In [80]:
def TL_extract_features(addr, driver, writer):
    driver.get("https://www.technolife.ir"+addr)
    sleep(1)
    spec_button = None
    spec_more_button = None
    while True:
        try:
            driver.execute_script(
                'window.scrollTo(0, document.body.scrollHeight);')
            sleep(1)
            driver.execute_script(
                'window.scrollTo(0, -document.body.scrollHeight);')
            spec_button = driver.find_element(By.ID, "product-info").find_elements(
                By.XPATH, "./..")
            sleep(1)
            driver.execute_script(
                "arguments[0].scrollIntoView();", spec_button[0])
            sleep(1)
            spec_more_button = spec_button[0].find_element(
                By.TAG_NAME, "button")
            spec_more_button.click()
            sleep(1)
            break
        except (NoSuchElementException, ElementClickInterceptedException):
            continue
    page = driver.page_source
    soup = bs(page, 'html.parser')

    global SUB_FEATURES
    scraped_sub_features = dict()
    features_div = soup.find("div", {
                             "id": "product-info"}).find("section")
    for fa_feature, eng_feature in (SUB_FEATURES.items()):
        if features_div.find(string=fa_feature+" :"):
            found = features_div.find(
                string=fa_feature+" :").parent.parent.findAll('p')[1:]
            scraped_sub_features[eng_feature] = "\r\n".join(
                [s.text for s in found])
        else:
            scraped_sub_features[eng_feature] = ""

    colors_element = soup.find("div", {"class": "flex lg:flex-wrap overflow-x-auto px-5 lg:px-0"}
                               ).findAll("div", {"class": "whitespace-nowrap"})
    colors = []
    for color in colors_element:
        colors.append(color.text)
    scraped_sub_features['Available Colors'] = colors

    score = soup.findAll("p", {"class": "mr-1 text-body-2"})
    if score:
        scraped_sub_features["Score"] = score[0].text

    brand = soup.findAll("a", {"data-cro-id": "pdp-breadcrumb-down"})
    if brand:
        scraped_sub_features["Brand"] = brand[0].text

    sellers = (soup.find("div", {"id": "sellerSection"}))
    if not sellers:
        price_no_discount = soup.findAll(
            "span", {"data-testid": "price-no-discount"})
        for element in price_no_discount:
            if element and " ".join(element.parent.parent.parent.attrs["class"][0:2]) == "flex items-center":
                scraped_sub_features["Price No Discount"] = element.text
                break

        price_final = soup.findAll("span", {"data-testid": "price-final"})
        for element in price_final:
            if element and " ".join(element.parent.parent.parent.attrs["class"][0:2]) == "flex items-center":
                scraped_sub_features["Price Final"] = element.text
                break

        Warranty = soup.findAll(string=re.compile("گارانتی"))
        if Warranty:
            scraped_sub_features["Warranty"] = Warranty[0].parent.text

        scraped_sub_features["Seller"] = soup.find('h3', string=re.compile("فروشنده")).parent.parent.findChildren(
            "div", recursive=False)[2].findChildren("p", recursive=True)[0]
        writer.writerow(list(scraped_sub_features.values()))
        return

    sellers_button = driver.find_element(By.ID, "sellerSection")
    driver.execute_script(
        "arguments[0].scrollIntoView();", sellers_button)
    sleep(1)
    sellers_more_button = sellers_button.find_elements(
        By.TAG_NAME, "span")
    if sellers_more_button:
        sellers_more_button[-1].click()

    page = driver.page_source
    soup = bs(page, 'html.parser')
    sellers = (soup.find("div", {"id": "sellerSection"}))
    for seller in sellers.findChildren("div", recursive=False)[0].\
            findChildren("div", recursive=False)[1:]:
        seller_div = seller.findChildren("div", recursive=False)[
            0].findChildren("div", recursive=False)
        name = seller_div[0].findChildren("div", recursive=False)[1]
        scraped_sub_features["Seller"] = name.findChildren(
            "div", recursive=False)[1].findChildren("div", recursive=False)[0].text
        warranty = seller_div[0].findChildren("div", recursive=False)[3]
        scraped_sub_features["Warranty"] = warranty.text
        price = seller_div[1].findChildren(
            "div", recursive=False)[0].findChildren(
            "div", recursive=False)[0].findChildren(
            "div", recursive=False)[0]
        prices = price.findChildren(
            "span", recursive=True)
        scraped_sub_features["Price No Discount"] = prices[-1].text
        if len(prices) == 3:
            scraped_sub_features["Price No Discount"] = prices[0].text
            scraped_sub_features["Price Final"] = prices[1].text
        writer.writerow(list(scraped_sub_features.values()))

In [46]:
def TL_scrape_list_of_mobiles(number_of_pages):
    driver = webdriver.Chrome()
    mobiles = []
    i = 1
    while i <= number_of_pages:
        driver.get(
            f'https://www.technolife.ir/product/list/69_800_801/%D8%AA%D9%85%D8%A7%D9%85%DB%8C-%DA%AF%D9%88%D8%B4%DB%8C%E2%80%8C%D9%87%D8%A7?page={i}&only_available=true')
        wait = WebDriverWait(driver, 60)
        wait.until(EC.presence_of_element_located((
            By.XPATH, "//div[@class=' h-full w-full flex-grow']")))
        page = driver.page_source
        soup = bs(page, 'html.parser')
        table = soup.find(
            'div', {'class': 'h-full w-full flex-grow'})
        mobiles.extend(table.findChildren("section"))
        i += 1
    driver.close()
    return mobiles

In [47]:
def TL_scrape_all_mobiles(file_addr, mobiles, start):
    driver = webdriver.Chrome()
    csv_file = open(file_addr, mode='a', encoding='utf-8')
    csv_writer = csv.writer(csv_file)
    i = start
    while i < len(mobiles):
        try:
            TL_extract_features(mobiles[i].find(
                'a')['href'], driver, csv_writer)
            i += 1
        except (NoSuchElementException, ElementClickInterceptedException):
            print(i)
        except Exception:
            print(i)
            driver.close()
            csv_file.close()
            raise
    driver.close()
    csv_file.close()

In [28]:
TL_mobiles = TL_scrape_list_of_mobiles(9)

In [42]:
FILE_ADDR = 'technolife_phones.csv'

In [43]:
make_new_csv(FILE_ADDR)

In [81]:
TL_scrape_all_mobiles(FILE_ADDR, TL_mobiles, 0)

0


AttributeError: 'NoneType' object has no attribute 'findAll'